In [ ]:
from flask import Flask, jsonify, json,g,request
import csv,json

csvFilePath= "world_universities_and_domains.csv"  
jsonFilePath= "world_universities_and_domainss.json"

data={}
with open(csvFilePath) as csvFile:
    csvReader =csv.DictReader(csvFile)
    for csvRow in csvReader:
        web_pages=csvRow["web_pages"]
        data[web_pages] = csvRow
        
with open(jsonFilePath,"w") as jsonFile:
    jsonFile.write(json.dumps(data, indent=4))

with open('world_universities_and_domains.json',encoding='utf-8') as f: 
    all_records = json.load(f)

app = Flask(__name__)

@app.route('/records', methods=['GET']) 
def get_all_records(): 
    return jsonify(all_records), 200

#Retrieve Web pages of all universities
@app.route('/records/webpages', methods=['GET']) 
def get_webpages_uni():
    name = ["NAME:{0} WEB_PAGES:{1}".format(x['name'], x['web_pages']) for x in all_records ]
    return jsonify(name), 200   
    
#Retrieve universities of all countries
@app.route('/records/country', methods=['GET']) 
def get_country_uni():
    name = ["COUNTRY:{0} \t\t\t NAME:{1}".format(x['country'], x['name']) for x in all_records ]
    return jsonify(name), 200

#Retrieve universities of a required country
@app.route('/records/<countryname>', methods=['GET']) 
def get_uni_by_country(countryname): 
    name = [x['name'] for x in all_records if x['country'] == countryname ]
    if len(name)==0: 
        return jsonify({'error':'Country name not found!'}), 404
    else:
        return jsonify(name), 200
    
@app.route('/records', methods=['POST']) 
def create_a_record(): 
    if not request.json or not 'name' in request.json: 
        return jsonify({'error':'the new record needs to have a University name'}), 400 
    new_record = {"alpha_two_code":request.json.get(""),"country": request.json.get(""),"domains":request.json.get(""),"name": request.json.get(""),"state-province":request.json.get(""),"web_pages":request.json.get("")} 
    all_records.append(new_record) 
    return jsonify({'message': 'created: /records/{}'}), 201
    #return jsonify({'message': 'created: /records/{}'.format(new_record['name'])}), 201
    
@app.route('/records/<universityname>', methods=['DELETE']) 
def delete_a_university(universityname): 
    matching_records = [x for x in all_records if x['name'] == universityname] 
    if len(matching_records)==0: 
        return jsonify({'error':'university name not found!'}), 404 
    all_records.remove(matching_records[0]) 
    return jsonify({'success': True})


if __name__ == '__main__': 
    app.run()
